#Naive Bayes


In [1]:
import os.path
baseDir = os.path.join('Data')
inputPath = os.path.join('Aula04', 'dac_sample.txt')
fileName = os.path.join(baseDir, inputPath)

if os.path.isfile(fileName):
    rawData = (sc
               .textFile(fileName, 4)
               .map(lambda x: x.replace('\t', ',')))  # work with either ',' or '\t' separated data
    print rawData.take(1)

[u'0,1,1,5,0,1382,4,15,2,181,1,2,,2,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,,3a171ecb,c5c50484,e8b83407,9727dd16']


In [2]:
# EXERCICIO
weights = [.8, .1, .1]
seed = 42
# Use randomSplit with weights and seed
rawTrainData, rawValidationData, rawTestData = rawData.randomSplit(weights, seed)
# Cache the data
rawTrainData.cache()
rawValidationData.cache()
rawTestData.cache()

nTrain = rawTrainData.count()
nVal = rawValidationData.count()
nTest = rawTestData.count()
print nTrain, nVal, nTest, nTrain + nVal + nTest
print rawData.take(5)

80139 9949 9912 100000
[u'0,1,1,5,0,1382,4,15,2,181,1,2,,2,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,,3a171ecb,c5c50484,e8b83407,9727dd16', u'0,2,0,44,1,102,8,2,2,4,1,1,,4,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,,3a171ecb,43f13e8b,e8b83407,731c3655', u'0,2,0,1,14,767,89,4,2,245,1,3,3,45,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,,,e587c466,ad3062eb,3a171ecb,3b183c5c,,', u'0,,893,,,4392,,0,0,0,,0,,,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,,,6b3a5ca6,,3a171ecb,9

In [3]:
# EXERCICIO
def parsePoint(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    return point.split(",")
    

#preparando as bases, separando os valores utilizando o separador ","
parsedTrainFeat = rawTrainData.map(parsePoint)
parsedValidationFeat = rawValidationData.map(parsePoint)
parsedTestFeat = rawTestData.map(parsePoint)


#funcao para enumerar as bases
def makeReady(Base):
    """
    Args:
        Base: Uma lista no seguinte formato
                [(label, feature1, feature3,..., featuren), (label2, feature1, feature2,..., featuren), etc]
        
        Returns: Retorna uma base enumerada da seguinte forma
                [(label1, (1, feature1), (2, feature2), ..., (n, featuren)), 
                ((label2, (1, feature1), (2, feature2), ..., (n, featuren)), etc]
    
    
    """
    return (Base
                .map(lambda x: list(enumerate(x)))
                .map(lambda x: (int(x[0][1]), x[1:])))
   

    
#bases prontas para serem usadas
readyTrainRDD = makeReady(parsedTrainFeat)
readyValidationRDD = makeReady(parsedValidationFeat)
readyTestRDD = makeReady(parsedTestFeat)


#criar o dicionário de palavras. 
#junta todas as features e reduz para que contenha apenas uma de cada
#tipo
categoriasRDD = (readyTrainRDD
                 .map(lambda (x,y): y)
                .flatMap(lambda x: x)
                .map(lambda (x,y): ((x,y),1))
                .reduceByKey(lambda x,y: x)
                .map(lambda ((x,y),z): (x,y))
                )

#coloca índice em cada feature para que cada uma possa ter um valor 
#único
orderedCategorias = sc.parallelize(sorted(categoriasRDD.collect()))
orderedZipCategorias = orderedCategorias.zipWithIndex()


#cria dicionario e calcula número de categorias
dictCategorias = dict(orderedZipCategorias.collect())
numCategorias = len(dictCategorias)

In [19]:
from pyspark.mllib.linalg import SparseVector

def oneHotEncoding(rawFeats, OHEDict, numOHEFeats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted.

    Args:
        rawFeats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sampleOne)
        OHEDict (dict): A mapping of (featureID, value) to unique integer.
        numOHEFeats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length numOHEFeats with indicies equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    #return SparseVector(numOHEFeats, [OHEDict.get(rawFeats[i],0.0) for i in range(len(rawFeats))], [1 for i in range(len(rawFeats))])
    return SparseVector(numOHEFeats, [OHEDict[rawFeats[i]] for i in range(len(rawFeats))], [1 for i in range(len(rawFeats))])


In [20]:
#funçao para transformar RDD em um vetor que pode ser usado para calcular as probabilidades
def makeVector(readyBase):
    """
        Args: 
            readyBase: base no formato 
                [(label1, (1, feature1), (2, feature2), ..., (n, featuren)), 
                ((label2, (1, feature1), (2, feature2), ..., (n, featuren)), etc]
        Returns:
            retorna uma RDD de tuplas, cujo primeiro valor é uma label e o segundo é um 
            SparseVector informando as posições das features encontradas
    """
    return readyBase.map(lambda x: (x[0], oneHotEncoding(x[1], dictCategorias, numCategorias)))


#transforma as bases prontas para poderem ser utilizadas
vectorTrainRDD = makeVector(readyTrainRDD)
vectorValidationRDD = makeVector(readyValidationRDD)
vectorTestRDD = makeVector(readyTestRDD)



In [8]:
"""
    Funcoes que calculam as probabilidades considerando a classe escolhida e 
    os atributos. (Tratados acima como label e features respectivamentes)
    
    Argumentos em comum:
        vectorTrain: RDD no formato (label/classe, SparseVector), onde o SparseVector informa quais 
                atributos a entrada contém
            
        classNum: O valor para a classe, que no caso pode ser 1 ou 0. Onde 1 corresponde ao ad clicado
                e 0 corresponde ao ad não clicado.
                
        attrib: Atributos considerados para calcular a possibilidade de ter um clique ou não
        
        nAttrib: quantidade de atributos sendo considerados no cálculo. Valor inicia com 
                o valor 1 e é utilizado para evitar uma multiplicação por 0 dado um 
                atributo que nunca apareceu na base de treinos.
"""

#funcao que retorna a probabilidade de um atributo dado uma classe
def probAttribClass(vectorTrain, classNum, attrib, nAttrib=1):
    classCount = vectorTrain.filter(lambda (x,y): x==classNum).count() 
    attriClassCount = vectorTrain.filter(lambda (x,y): x==classNum and y[attrib]==1).count()
    return (attriClassCount+1)/float(classCount+nAttrib)

#funcao que retorna a probabilidade de uma classe
def probClass(vectorTrain, classNum, nAttrib=1):
    classCount = vectorTrain.filter(lambda (x,y): x==classNum).count()
    nTrain = vectorTrain.count()
    return (classCount+1)/float(nTrain+nAttrib)

#funcao que retorna a probabilidade de um atributo
def probAttrib(vectorTrain, attrib, nAttrib=1):
    attribCount = vectorTrain.filter(lambda (x,y): y[attrib]==1).count()
    nTrain = vectorTrain.count()
    return (attribCount+1)/float(nTrain+nAttrib)

#funcao que retorna a probabilidade de uma classe dados vários atributos
def probClassAttrib(vectorTrain, classNum, attribList):
    probList = map(lambda x: (probAttribClass(vectorTrain, classNum, x)*probClass(vectorTrain, classNum))/probAttrib(vectorTrain, x), attribList)
    return reduce(lambda x,y: x*y, probList)

#funcao teste que mostra as probabilidades sem a multiplicação entre elas
def probClassAttribTest(vectorTrain, classNum, attribList):
    nAttrib = len(attribList)
    return map(lambda x: (probAttribClass(vectorTrain, classNum, x, nAttrib)*probClass(vectorTrain, classNum, nAttrib))/probAttrib(vectorTrain, x, nAttrib), attribList)



In [ ]:
#Testing

#print vectorTrainRDD.take(2)

#print probClassAttribTest(vectorValidationRDD, 1, [2, 3, 4])
print probClassAttrib(vectorTrainRDD, 0, [2, 3, 4])
